In [1]:
import numpy as np

file1 = open('Lab3_grid1.txt', 'r') 
(W, H) = [t(s) for t,s in zip((int,int),file1.readline().split())]
noise = [t(s) for t,s in zip((float,str),file1.readline().split())][0]
immediate_rewards = [t(s) for t,s in zip((float,str),file1.readline().split())][0]

N = W*H #total number of states

# ---------------- Task 4 change ----------------
NA = 8 # number of actions (4 cardinal + 4 diagonal)
Directions = np.array(['L','R','U','D','UL','UR','DL','DR'])
# ----------------------------------------------

T = np.zeros((N,N,NA)) #state transition probabilities
Rewards = np.zeros((N,1)) + immediate_rewards #immediate rewards
Terminals = np.zeros((N,1)) #terminal states
Walls = np.zeros((N,1)) #(internal) wall states

prob1 = 1-noise
prob2 = noise/2

while True: 
    line = file1.readline() 
    if not line: 
        break
    if line.find('T') != -1:
        (_,x, y, r) = [t(s) for t,s in zip((str,int,int,float),line.split())]
        Terminals[y*W+x] = 1
        Rewards[y*W+x] = r
    if line.find('W') != -1:
        (_,x, y) = [t(s) for t,s in zip((str,int,int),line.split())]
        Walls[y*W+x] = 1     
file1.close()

for i in range(W):
    for j in range(H):

        # ---------------- Existing 4-action transitions (UNCHANGED) ----------------

        if i+1<W and Walls[j*W+i+1]==0: #not wall on the right
            T[j*W+i,j*W+i+1,1] += prob1 
            T[j*W+i,j*W+i+1,2] += prob2
            T[j*W+i,j*W+i+1,3] += prob2
        else:
            T[j*W+i,j*W+i,1] += prob1 
            T[j*W+i,j*W+i,2] += prob2
            T[j*W+i,j*W+i,3] += prob2

        if i-1>=0 and Walls[j*W+i-1]==0: #not wall on the left
            T[j*W+i,j*W+i-1,0] += prob1 
            T[j*W+i,j*W+i-1,2] += prob2
            T[j*W+i,j*W+i-1,3] += prob2
        else:
            T[j*W+i,j*W+i,0] += prob1 
            T[j*W+i,j*W+i,2] += prob2
            T[j*W+i,j*W+i,3] += prob2

        if j+1<H and Walls[(j+1)*W+i]==0: #not wall below
            T[j*W+i,(j+1)*W+i,3] += prob1 
            T[j*W+i,(j+1)*W+i,0] += prob2
            T[j*W+i,(j+1)*W+i,1] += prob2
        else:
            T[j*W+i,j*W+i,3] += prob1 
            T[j*W+i,j*W+i,0] += prob2
            T[j*W+i,j*W+i,1] += prob2       

        if j-1>=0 and Walls[(j-1)*W+i]==0: #not wall above
            T[j*W+i,(j-1)*W+i,2] += prob1 
            T[j*W+i,(j-1)*W+i,0] += prob2
            T[j*W+i,(j-1)*W+i,1] += prob2
        else:
            T[j*W+i,j*W+i,2] += prob1 
            T[j*W+i,j*W+i,0] += prob2
            T[j*W+i,j*W+i,1] += prob2    

        # ---------------- Task 4: Diagonal actions ----------------
        s = j*W + i

        # UL (action 4): intended UL, noise → U, L
        if j-1>=0 and i-1>=0 and Walls[(j-1)*W + (i-1)]==0:
            T[s,(j-1)*W + (i-1),4] += prob1
        else:
            T[s,s,4] += prob1

        if j-1>=0 and Walls[(j-1)*W + i]==0:
            T[s,(j-1)*W + i,4] += prob2
        else:
            T[s,s,4] += prob2

        if i-1>=0 and Walls[j*W + (i-1)]==0:
            T[s,j*W + (i-1),4] += prob2
        else:
            T[s,s,4] += prob2


        # UR (action 5): intended UR, noise → U, R
        if j-1>=0 and i+1<W and Walls[(j-1)*W + (i+1)]==0:
            T[s,(j-1)*W + (i+1),5] += prob1
        else:
            T[s,s,5] += prob1

        if j-1>=0 and Walls[(j-1)*W + i]==0:
            T[s,(j-1)*W + i,5] += prob2
        else:
            T[s,s,5] += prob2

        if i+1<W and Walls[j*W + (i+1)]==0:
            T[s,j*W + (i+1),5] += prob2
        else:
            T[s,s,5] += prob2


        # DL (action 6): intended DL, noise → D, L
        if j+1<H and i-1>=0 and Walls[(j+1)*W + (i-1)]==0:
            T[s,(j+1)*W + (i-1),6] += prob1
        else:
            T[s,s,6] += prob1

        if j+1<H and Walls[(j+1)*W + i]==0:
            T[s,(j+1)*W + i,6] += prob2
        else:
            T[s,s,6] += prob2

        if i-1>=0 and Walls[j*W + (i-1)]==0:
            T[s,j*W + (i-1),6] += prob2
        else:
            T[s,s,6] += prob2


        # DR (action 7): intended DR, noise → D, R
        if j+1<H and i+1<W and Walls[(j+1)*W + (i+1)]==0:
            T[s,(j+1)*W + (i+1),7] += prob1
        else:
            T[s,s,7] += prob1

        if j+1<H and Walls[(j+1)*W + i]==0:
            T[s,(j+1)*W + i,7] += prob2
        else:
            T[s,s,7] += prob2

        if i+1<W and Walls[j*W + (i+1)]==0:
            T[s,j*W + (i+1),7] += prob2
        else:
            T[s,s,7] += prob2
